# Basics of Features at M0

This notebook walks through some core concepts around Features (M1 elements that are interpreted as sequences of atoms of length at least two) and how they relate to Classifiers (M1 elements that are interpreted as sequences of atoms of length at least one).

## Determining the Minimum Length of Feature Sequence

The minimum length of a feature sequence is based on how many levels of nesting it has within other types. 

The "build sequence templates" function is used to generate the minimal sequences for a given feature based on its nesting.

In [ ]:
from pathlib import Path
import networkx as nx

import pymbe.api as pm

from pymbe.client import SysML2Client
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.interpretation import repack_instance_dictionaries
from pymbe.interpretation.interp_playbooks import (
    build_expression_sequence_templates,
    build_sequence_templates,
    random_generator_playbook,
)
from pymbe.interpretation.results import *
from pymbe.label import get_label_for_id
from pymbe.query.metamodel_navigator import feature_multiplicity
from pymbe.query.query import (
    roll_up_multiplicity,
    roll_up_upper_multiplicity,
    roll_up_multiplicity_for_type,
    get_types_for_feature,
    get_features_typed_by_type,
)
from pymbe.local.stablization import build_stable_id_lookups

## Examples for Minimum Lengths

We can look at some examples here to show how this should work. The first of these is the Simple Parts Test.

In [ ]:
parts_client = SysML2Client()

simple_parts_file = Path(pm.__file__).parent / "../../tests/fixtures/Simple Parts Model.json"

parts_client._load_from_file(simple_parts_file)

parts_lpg = SysML2LabeledPropertyGraph()
parts_lpg.model = parts_client.model

SIMPLE_MODEL = "Model::Simple Parts Model::"
FAKE_LIBRARY = "Model::Simple Parts Model::Fake Library::"

[id_to_parts_name_lookup, parts_name_to_id_lookup] = build_stable_id_lookups(parts_lpg)

parts_lpg.model.MAX_MULTIPLICITY = 10

In [ ]:
power_source_id = parts_name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::Power Source: Part <<PartUsage>>"]
power_user_id = parts_name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::Power User: Part <<PartUsage>>"]
power_in_id = parts_name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::Power User: Part::Power In: Port <<PortUsage>>"]
power_out_id = parts_name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::Power Source: Part::Power Out: Port <<PortUsage>>"]
connect_use_id = parts_name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::powerToUser: Connection <<ConnectionUsage>>"]
power_group_id = parts_name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part <<PartUsage>>"]

### Simple Parts Example

#### Feature names for sequences

The sequence templates below are built to show where the Features are nested within each other and the minimum length of a given sequence.

In [ ]:
feature_templates_with_ids = [[item for item in seq] for seq in build_sequence_templates(parts_lpg)
                                    if id_to_parts_name_lookup[seq[0]].startswith("Model::Simple Parts Model")]
feature_templates_with_names = [[id_to_parts_name_lookup[item] for item in seq] for seq in build_sequence_templates(parts_lpg)
                                    if id_to_parts_name_lookup[seq[0]].startswith("Model::Simple Parts Model")]
feature_templates_with_names

In [ ]:
model_package = parts_lpg.model.ownedElement["Simple Parts Model"]

#### Feature types for sequences

We can also inspect the types of each of the features (or classifier themselves) to see what will be placed into each step of the sequence.

In the below, what we mean to say is that for a given position, the atom in that place will also appear in the 1-tail of the given type. For example, the type that corresponds to the Power User: Part::Power In: Port in the second position, which is a Part, will have all atoms in the 1-tail of Part's sequences. 

In [ ]:
[
    [
        [id_to_parts_name_lookup[typ] for typ in get_types_for_feature(parts_lpg, parts_lpg.model.elements[item])]
    for item in seq
    ]
for seq in feature_templates_with_ids
]

#### Counting multiplicty for types

Once we know the types of all the features (and can run this query in the opposite direction), we can count how many times a given type will be used in our sequences.

In [ ]:
[id_to_parts_name_lookup[item] for item in get_features_typed_by_type(parts_lpg, parts_lpg.model.ownedElement["Parts"].ownedElement["Part"])]

In [ ]:
[parts_lpg.model.elements[item].isAbstract for item in
     get_features_typed_by_type(parts_lpg, parts_lpg.model.ownedElement["Parts"].ownedElement["Part"])]

In [ ]:
[roll_up_multiplicity(parts_lpg, parts_lpg.model.elements[item], "upper") for item in
     get_features_typed_by_type(parts_lpg, parts_lpg.model.ownedElement["Parts"].ownedElement["Part"])]

In [ ]:
[feature_multiplicity(parts_lpg.model.elements[item], "upper") for item in
     get_features_typed_by_type(parts_lpg, parts_lpg.model.ownedElement["Parts"].ownedElement["Part"])]

In [ ]:
roll_up_multiplicity_for_type(parts_lpg, parts_lpg.model.ownedElement["Parts"].ownedElement["Part"], "lower")

In [ ]:
roll_up_multiplicity_for_type(parts_lpg, parts_lpg.model.ownedElement["Parts"].ownedElement["Part"], "upper")